In [8]:
%load_ext autoreload

from utils.preprocessing import process_seqs
from model.train_model import train_siamese_model
from model.models_cstm import get_embedding_model
import pickle

def test_split(seqs_path, y_path, depth):
    # ((X_train, X_test, X_val), (y_train, y_test, y_val)) = process_seqs(
    data = process_seqs(
        seqs_path=seqs_path,
        train_test_split='distance',
        split_depth=depth,
        test_size=0.2,
        val_size=0.2,
        load_y=y_path,
        lim=1000,
        verbose=True
    )
    in_dim = data[0][0].shape[1] # size of sequence
    out_dim = int(in_dim / 2)
    history = train_siamese_model(
        data,
        embedding_model=get_embedding_model(in_dim=in_dim, out_dim=out_dim),
        distance_metric='euclidean',
        epochs=1,
        batch_size=512
    )
    return history

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
phes_fasta_path = "./data/phes/phes_na.fa"
phes_y_path = "./data/phes/y.pkl"
history_path="./results/train_test"

In [ ]:
for depth in range(12):
    print(f"DEPTH = {depth}: ########################")
    history = test_split(phes_fasta_path, phes_y_path, 0)
    with open(f"{history_path}/history_{depth}.pkl", "wb") as f:
        pickle.dump(history, f)

DEPTH = 0: ########################
Reading inputs...
	Done in 0.560 seconds
	Shape of X: (1000, 1674)
Loading distances from ./data/phes/y.pkl
Splitting X values...
	Done in 0.177 seconds
	Shapes of data: (600, 1674), (200, 1674), (200, 1674)
 28/350 [=>............................] - ETA: 1:20 - loss: 44308.7695